In [1]:
import tensorflow as tf
import numpy as np

fd = open('./Desktop/mnist/train-images.idx3-ubyte')
images = np.fromfile(file=fd, dtype=np.uint8)
images = images[16:].reshape([60000,28,28]).astype(np.float)
images = images/127 - 1.0

fd = open('./Desktop/mnist/train-labels.idx1-ubyte')
labels = np.fromfile(file=fd, dtype=np.uint8)
labels = labels[8:].reshape([60000]).astype(np.int)


In [2]:
fd = open('./Desktop/mnist/t10k-images.idx3-ubyte')
images2 = np.fromfile(file=fd, dtype=np.uint8)
images2 = images2[16:].reshape([10000,28,28]).astype(np.float)
images2 = images2/127 - 1.0

fd = open('./Desktop/mnist/t10k-labels.idx1-ubyte')
labels2 = np.fromfile(file=fd, dtype=np.uint8)
labels2 = labels2[8:].reshape([10000]).astype(np.int)

In [3]:
type(images)
print(images.shape)
print(labels.shape)

(60000, 28, 28)
(60000,)


In [5]:
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name='X')
Y = tf.placeholder(shape=[None], dtype=tf.int64, name='Y')
is_training=tf.placeholder(tf.bool)

# hidden1
weights1 = tf.Variable(tf.truncated_normal([28*28, 300], stddev=0.02), name="weights1")
biases1 = tf.Variable(tf.zeros([300]), name="biases1")
hidden1 = tf.matmul(X, weights1) + biases1
hidden1 = tf.nn.relu(hidden1, "relu1")
hidden1 = tf.layers.dropout(hidden1, rate=0.5, training=is_training)

# hidden2
weights2 = tf.Variable(tf.truncated_normal([300, 200], stddev=0.02), name="weights2")
biases2 = tf.Variable(tf.zeros([200]), name="biases2")
hidden2 = tf.matmul(hidden1, weights2) + biases2
hidden2 = tf.nn.relu(hidden2, "relu2")
hidden2 = tf.layers.dropout(hidden2, rate=0.5, training=is_training)

# ouput
weights3 = tf.Variable(tf.truncated_normal([200, 10]),name="weights3")
biases3 = tf.Variable(tf.truncated_normal([10]), name="biases3")

Y_= tf.matmul(hidden2,weights3) + biases3

Y_onehot = tf.one_hot(Y, 10, axis=1)

softmax = tf.nn.softmax(Y_)

correct = tf.equal(tf.argmax(softmax,1), tf.argmax(Y_onehot,1))
acc = tf.reduce_mean(tf.cast(correct, tf.float32))

#Creat Model
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_, labels=Y_onehot))

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
optimize = trainer.minimize(loss)

# In training loop

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    batch_size= 1000
    batch_count = 60000 // batch_size   
    
    for epoch in range(100):
        total_loss = 0
        for i in range(batch_count):
            batch_index = i * batch_size
            img = np.reshape(images[batch_index:batch_index+batch_size], [batch_size, 28*28])
            label = labels[batch_index:batch_index+batch_size]
            _, loss_v = sess.run([optimize, loss],feed_dict={X:img, Y:label,is_training:True})
            total_loss += loss_v 
            
        print(total_loss)
        
    TRAINING_ACC = sess.run(acc, feed_dict={X:img, Y:label, is_training:False})
    print(TRAINING_ACC)
    
    TEST_ACCURACY_ACC = sess.run(acc, feed_dict={X:np.reshape(images2,[10000,28*28]), Y:labels2, is_training:False})
    print(TEST_ACCURACY_ACC)


130.515475154
106.446551323
82.6016764641
61.6039461493
50.4490494728
41.1024177372
37.3117028773
33.7022499442
31.8176181912
29.4912424088
28.2710841894
27.4117198288
26.2585771084
24.7857532203
24.2314069867
23.3149623424
22.8530297726
22.0555340052
22.0324048996
21.566193521
20.9482327998
20.1378663331
19.9083580077
19.4153202027
19.5983663499
18.7194954008
18.495348379
18.2106423378
17.8332217485
17.7451429963
17.5271548331
17.2132936865
16.9329973757
16.8106015772
16.4854656905
16.6250072718
16.1277476251
15.9224317521
15.6776892096
15.6122386456
15.7460410893
15.1387173533
15.121439755
14.9520377517
14.9405332357
14.5772845894
14.7777748853
14.2413452417
14.2860784084
14.148756519
14.0167483836
14.1068804115
13.8233779818
13.7932274044
13.5412774086
13.373561658
13.3949804455
13.4761906415
13.2823494375
13.0617136583
12.8612490892
12.9238018543
12.9171699509
13.1076461524
12.791732505
12.7395332083
12.625034593
12.367959246
12.2851676196
12.0257522017
12.3522628546
12.0990589261
